In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import cv2 
import os
import torch.onnx


In [2]:
# only torchvision models are available
model = torch.load('FUll_model_weight_AMD.pth')     # Load the model
model.eval() # Set the model to evaluate mode

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
# define the directory for further converted model save
onnx_model_path = "onnx_models"
# define the name of further converted model
onnx_model_name = "KeyNet_v2.onnx"
# create directory for further converted model
os.makedirs(onnx_model_path, exist_ok=True)
# get full path to the converted model
full_model_path = os.path.join(onnx_model_path, onnx_model_name)
# generate model input
generated_input = torch.randn(1, 3, 672, 672).cuda()

# model export into ONNX format
torch.onnx.export(
    model,
    generated_input,
    full_model_path,
    verbose=True,
    input_names=["input"],
    output_names=["output"],
    opset_version=11
)

graph(%input : Float(1, 3, 672, 672, strides=[1354752, 451584, 672, 1], requires_grad=0, device=cuda:0),
      %fc.weight : Float(2, 2048, strides=[2048, 1], requires_grad=1, device=cuda:0),
      %fc.bias : Float(2, strides=[1], requires_grad=1, device=cuda:0),
      %497 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cuda:0),
      %498 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %500 : Float(64, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cuda:0),
      %501 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %503 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cuda:0),
      %504 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %506 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cuda:0),
      %507 : Float(256, strides=[1], requires_grad=0, device=cuda:0),
      %509 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cuda:0),
      %510 

In [ ]:
opencv_net = cv2.dnn.readNetFromONNX("onnx_models/KeyNet_v2.onnx")
labels = ['NON_AMD', 'AMD']
# read the image
test_image =  r'C:\Users\bahadur\Downloads\GUM\orginal\train\Negative\negative (2).JPG'
input_img = cv2.imread(test_image, cv2.IMREAD_COLOR)
input_img = input_img.astype(np.float32)
input_img = cv2.resize(input_img, (672, 672))
# define preprocess parameters
mean = np.array([0.485, 0.456, 0.406]) * 255.0
scale = 1 / 255.0
std = [0.229, 0.224, 0.225]
# prepare input blob to fit the model input:
# 1. subtract mean
# 2. scale to set pixel values from 0 to 1
input_blob = cv2.dnn.blobFromImage(
    image=input_img,
    scalefactor=scale,
    size=(672, 672),  # img target size
    mean=mean,
    swapRB=True,  # BGR -> RGB
    crop=True  # center crop
)
# 3. divide by std
input_blob[0] /= np.asarray(std, dtype=np.float32).reshape(3, 1, 1)
# set OpenCV DNN input
opencv_net.setInput(input_blob)
# OpenCV DNN inference
out = opencv_net.forward()
print("OpenCV DNN prediction: \n")
print("* shape: ", out.shape)
# get the predicted class ID
imagenet_class_id = np.argmax(out)
# get confidence
confidence = out[0][imagenet_class_id]
print("* class ID: {}, label: {}".format(imagenet_class_id, labels[imagenet_class_id]))
print("* confidence: {:.4f}".format(confidence))

In [1]:
import os
import numpy as np
from flask import Flask, request , jsonify , abort , send_from_directory , make_response
from werkzeug.utils import secure_filename
import tempfile
import cv2

: 

: 